<a href="https://colab.research.google.com/github/AkashKoley012/Computer-Vision-Projects/blob/main/UTKFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 99% 329M/331M [00:15<00:00, 21.7MB/s]
100% 331M/331M [00:15<00:00, 22.2MB/s]


In [2]:
import zipfile
with zipfile.ZipFile('utkface-new.zip', 'r') as zip_ref:
    zip_ref.extractall('utkface')

In [3]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [4]:
os.remove('utkface-new.zip')

In [5]:
images = []
ages = []
genders = []
ethnics = []

for file in tqdm(os.listdir('/content/utkface/UTKFace')):
  # Split the filename by underscore and check if it has at least 3 parts
  parts = file.split('_')
  if len(parts) > 3:
    try:
      img = cv2.imread('/content/utkface/UTKFace/' + file)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      images.append(img)

      age, gender, ethnicity = parts[:3]
      ages.append(int(age))
      genders.append(int(gender))
      ethnics.append(int(ethnicity))
    except ValueError:
      print(f"Skipping file with unexpected format: {file}")
  else:
    print(f"Skipping file with unexpected format: {file}")

 13%|█▎        | 3146/23708 [00:01<00:06, 2977.44it/s]

Skipping file with unexpected format: 61_1_20170109150557335.jpg.chip.jpg


 92%|█████████▏| 21840/23708 [00:07<00:00, 3311.56it/s]

Skipping file with unexpected format: 39_1_20170116174525125.jpg.chip.jpg


100%|██████████| 23708/23708 [00:08<00:00, 2806.10it/s]

Skipping file with unexpected format: 61_1_20170109142408075.jpg.chip.jpg


In [6]:
print(len(images), len(ages), len(genders), len(ethnics))

23705 23705 23705 23705


In [7]:
df = pd.DataFrame({'image': images, 'age': ages, 'gender': genders, 'ethnicity': ethnics})

In [8]:
df.shape

(23705, 4)

In [9]:
split = int(len(df) * 0.8)
train_df = df[:split]
test_df = df[split:]

# Dataset & DataLoader

In [10]:
class UTKFaceDataset(Dataset):
  def __init__(self, df, transform=None):
    self.df = df
    self.transform = transform
  def __len__(self):
    return len(self.df)
  def __getitem__(self, idx):
    image = self.df.iloc[idx]['image']
    age = self.df.iloc[idx]['age']
    gender = self.df.iloc[idx]['gender']
    ethnicity = self.df.iloc[idx]['ethnicity']

    # Convert the NumPy array to a PIL Image before applying transforms
    image = Image.fromarray(image)

    if self.transform:
      image = self.transform(image)

    age = torch.tensor(age, dtype=torch.float32)
    gender = torch.tensor(gender, dtype=torch.float32)
    ethnicity = torch.tensor(ethnicity, dtype=torch.long)
    return image, age, gender, ethnicity

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
train_dataset = UTKFaceDataset(train_df, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = UTKFaceDataset(test_df, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [13]:
for images, age_labels, gender_labels, ethnicity_labels in train_dataloader:
  print(images.shape, age_labels.shape, gender_labels.shape, ethnicity_labels.shape)
  break

torch.Size([64, 3, 224, 224]) torch.Size([64]) torch.Size([64]) torch.Size([64])


In [14]:
for images, age_labels, gender_labels, ethnicity_labels in test_dataloader:
  print(images)
  break

tensor([[[[ 0.8447,  0.7248,  0.5707,  ..., -1.5528, -1.5357, -1.5185],
          [ 0.6734,  0.6049,  0.5193,  ..., -1.5528, -1.5357, -1.5357],
          [ 0.3309,  0.2796,  0.2453,  ..., -1.5528, -1.5357, -1.5357],
          ...,
          [ 0.6906,  0.7077,  0.7248,  ...,  0.2796,  0.2967,  0.2967],
          [ 0.6906,  0.7077,  0.7248,  ...,  0.2796,  0.2796,  0.2796],
          [ 0.7077,  0.7077,  0.7248,  ...,  0.2624,  0.2796,  0.2796]],

         [[ 0.1176, -0.0049, -0.1625,  ..., -1.6856, -1.6506, -1.5980],
          [-0.0574, -0.1275, -0.2150,  ..., -1.6856, -1.6681, -1.6155],
          [-0.3725, -0.4251, -0.4776,  ..., -1.6856, -1.6681, -1.6155],
          ...,
          [ 0.3803,  0.3978,  0.4153,  ..., -0.6176, -0.6001, -0.6001],
          [ 0.3803,  0.3978,  0.4153,  ..., -0.6176, -0.6176, -0.6176],
          [ 0.3978,  0.3978,  0.4153,  ..., -0.6352, -0.6176, -0.6176]],

         [[-0.0615, -0.1835, -0.3404,  ..., -1.6127, -1.5779, -1.5256],
          [-0.2358, -0.3055, -

# Model use ResNet-152

In [26]:
class AgeGenderEthnicityResNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = models.resnet152(pretrained=True)
    self.resnet.fc = nn.Identity()
    self.fc = nn.Linear(512, 512)

    self.age_output = nn.Linear(512, 1)
    self.gender_output = nn.Linear(512, 1)
    self.ethnicity_output = nn.Linear(512, 5)

  def forward(self, x):
    x = self.resnet(x)
    x = self.fc(x)
    age = self.age_output(x)
    gender = torch.sigmoid(self.gender_output(x))
    ethnicity = self.ethnicity_output(x)
    return age, gender, ethnicity


In [27]:
lr = 0.001

# Initialize Model
model = AgeGenderEthnicityResNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss functions
criterion_age = nn.MSELoss()
criterion_gender = nn.BCELoss()
criterion_ethnicity = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
epochs = 10

# Training Loop
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    for images, age_labels, gender_labels, ethnicity_labels in train_dataloader:
        images, age_labels, gender_labels, ethnicity_labels = (
            images.to(device),
            age_labels.to(device),
            gender_labels.float().unsqueeze(1).to(device),
            ethnicity_labels.to(device)
        )

        optimizer.zero_grad()

        age_preds, gender_preds, ethnicity_preds = model(images)
        loss_age = criterion_age(age_preds, age_labels)
        loss_gender = criterion_gender(gender_preds, gender_labels)
        loss_ethnicity = criterion_ethnicity(ethnicity_preds, ethnicity_labels)

        loss = loss_age + loss_gender + loss_ethnicity

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_dataloader)}")



  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 10%|█         | 1/10 [02:20<21:01, 140.12s/it]

Epoch [1/10], Loss: 413.6888554120305


 20%|██        | 2/10 [04:38<18:31, 138.89s/it]

Epoch [2/10], Loss: 402.0406140671033


 30%|███       | 3/10 [06:57<16:12, 138.91s/it]

...

Epoch [8/10], Loss: 400.6105963710181


 90%|█████████ | 9/10 [20:45<02:18, 138.27s/it]

Epoch [9/10], Loss: 399.35990561539876


100%|██████████| 10/10 [23:03<00:00, 138.31s/it]

Epoch [10/10], Loss: 399.3796923602069
